# Import Libraries

In [2]:
#Importing libraries needed
import pandas as pd
import numpy as np
from collections import Counter
import re 

# Open Dataset

In [2]:
df = pd.read_csv(r'C:\Users\salom\OneDrive\Área de Trabalho\DSFinal\IMDB Horror movies.csv')

In [3]:
df.head()

,Title,Genres,Release Date,Release Country,Movie Rating,Review Rating,Movie Run Time,Plot,Cast,Language,Filming Locations,Budget
0,Gut (2012),Drama| Horror| Thriller,26-Oct-12,USA,NaN,3.9,91 min,"Directed by Elias. With Jason Vail, Nicholas W...",Jason Vail|Nicholas Wilder|Sarah Schoofs|Kirst...,English,"New York, USA",NaN
1,The Haunting of Mia Moss (2017),Horror,13-Jan-17,USA,NaN,NaN,NaN,"Directed by Jake Zelch. With Nicola Fiore, Bri...",Nicola Fiore|Brinke Stevens|Curtis Carnahan|Ja...,English,NaN,"$30,000"
2,Sleepwalking (2017),Horror,21-Oct-17,Canada,NaN,NaN,NaN,"Directed by David Briggs. With Alysia Topol, A...",Alysia Topol|Anthony Makela|Kelsi Ashley|Patri...,English,"Sudbury, Ontario, Canada",NaN
3,Treasure Chest of Horrors II (2013),Comedy| Horror| Thriller,23-Apr-13,USA,NOT RATED,3.7,82 min,"Directed by M. Kelley, Shawn C. Phillips, Alex...",Veronica Ricci|Nicholas Adam Clark|James Culle...,English,"Baltimore, Maryland, USA",NaN
4,Infidus (2015),Crime| Drama| Horror,10-Apr-15,USA,NaN,5.8,80 min,"Directed by Giulio De Santi. With Bonini Mino,...",Bonini Mino|Massimo Caratelli|Maurizio Zaffino...,Italian,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3328 entries, 0 to 3327
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              3328 non-null   object 
 1   Genres             3327 non-null   object 
 2   Release Date       3328 non-null   object 
 3   Release Country    3328 non-null   object 
 4   Movie Rating       1451 non-null   object 
 5   Review Rating      3076 non-null   float64
 6   Movie Run Time     2784 non-null   object 
 7   Plot               3327 non-null   object 
 8   Cast               3314 non-null   object 
 9   Language           3257 non-null   object 
 10  Filming Locations  2096 non-null   object 
 11  Budget             1234 non-null   object 
dtypes: float64(1), object(11)
memory usage: 312.1+ KB


# Get Cast Frequency

In [5]:
# Split the cast column by the '|' character and flatten the result into a list
cast_list = df['Cast'].str.split('|').explode()

# Count the frequency of each cast member
cast_counts = Counter(cast_list)

# Convert the cast counts to a DataFrame
cast_counts_df = pd.DataFrame(list(cast_counts.items()), columns=['Cast Member', 'Frequency'])

# Sort by frequency in descending order
cast_counts_df = cast_counts_df.sort_values(by='Frequency', ascending=False)

# Reset the index for a cleaner DataFrame
cast_counts_df = cast_counts_df.reset_index(drop=True)

print(cast_counts_df)

                 Cast Member  Frequency
0              Lloyd Kaufman         29
1               Eric Roberts         28
2              Debbie Rochon         23
3                Maria Olsen         21
4             Brinke Stevens         18
...                      ...        ...
33877  Kathy Blaze Jefferson          1
33878            Liam Hawley          1
33879         Katie Hargrave          1
33880            Ariel Gomez          1
33881        Gabriel Daniels          1

[33882 rows x 2 columns]


In [6]:
df['Cast Member'] = cast_counts_df['Cast Member'] 
df['Cast Member Frequency'] = cast_counts_df['Frequency']

# Director

In [7]:
# Extract the "Directed by" part and director's name using a regular expression
df['Director'] = df['Plot'].str.extract(r'Directed by (.+?)\.')

# Remove the "Directed by" part from the original column
df['Description'] = df['Plot'].str.replace(r'Directed by .+?\.', '')

df.head()

C:\Users\salom\AppData\Local\Temp\ipykernel_20564\1749570473.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Description'] = df['Plot'].str.replace(r'Directed by .+?\.', '')


,Title,Genres,Release Date,Release Country,Movie Rating,Review Rating,Movie Run Time,Plot,Cast,Language,Filming Locations,Budget,Cast Member,Cast Member Frequency,Director,Description
0,Gut (2012),Drama| Horror| Thriller,26-Oct-12,USA,NaN,3.9,91 min,"Directed by Elias. With Jason Vail, Nicholas W...",Jason Vail|Nicholas Wilder|Sarah Schoofs|Kirst...,English,"New York, USA",NaN,Lloyd Kaufman,29,Elias,"With Jason Vail, Nicholas Wilder, Sarah Schoo..."
1,The Haunting of Mia Moss (2017),Horror,13-Jan-17,USA,NaN,NaN,NaN,"Directed by Jake Zelch. With Nicola Fiore, Bri...",Nicola Fiore|Brinke Stevens|Curtis Carnahan|Ja...,English,NaN,"$30,000",Eric Roberts,28,Jake Zelch,"With Nicola Fiore, Brinke Stevens, Curtis Car..."
2,Sleepwalking (2017),Horror,21-Oct-17,Canada,NaN,NaN,NaN,"Directed by David Briggs. With Alysia Topol, A...",Alysia Topol|Anthony Makela|Kelsi Ashley|Patri...,English,"Sudbury, Ontario, Canada",NaN,Debbie Rochon,23,David Briggs,"With Alysia Topol, Anthony Makela, Kelsi Ashl..."
3,Treasure Chest of Horrors II (2013),Comedy| Horror| Thriller,23-Apr-13,USA,NOT RATED,3.7,82 min,"Directed by M. Kelley, Shawn C. Phillips, Alex...",Veronica Ricci|Nicholas Adam Clark|James Culle...,English,"Baltimore, Maryland, USA",NaN,Maria Olsen,21,M,"Kelley, Shawn C. Phillips, Alex Powers. With ..."
4,Infidus (2015),Crime| Drama| Horror,10-Apr-15,USA,NaN,5.8,80 min,"Directed by Giulio De Santi. With Bonini Mino,...",Bonini Mino|Massimo Caratelli|Maurizio Zaffino...,Italian,NaN,NaN,Brinke Stevens,18,Giulio De Santi,"With Bonini Mino, Massimo Caratelli, Maurizio..."


# Title Count 

In [8]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import string

def remove_less_informative(text):
    words = word_tokenize(text)
    tagged_words = pos_tag(words)
    
    # Define parts of speech to be removed
    less_informative_pos = set(['DT', 'CC', 'IN', 'PRP', 'MD', 'CD'])
    
    # Define punctuation to be removed
    punctuation = set(string.punctuation)
    
    # Filter words based on parts of speech and punctuation
    filtered_words = [word for word, pos in tagged_words if pos not in less_informative_pos and word not in punctuation]
    return ' '.join(filtered_words)

# Apply the function to the DataFrame
df['Title'] = df['Title'].apply(remove_less_informative)

In [9]:
# Tokenize the titles into words (split by whitespace)
words = ' '.join(df['Title']).split()

# Remove punctuation and convert to lowercase (optional)
words = [re.sub(r'[^\w\s]', '', word).lower() for word in words]

# Count the frequency of each word
word_counts = Counter(words)

# Convert the word counts to a DataFrame
title_counts_df = pd.DataFrame(list(word_counts.items()), columns=['Word', 'Frequency'])

# Sort by frequency in descending order
title_counts_df = title_counts_df.sort_values(by='Frequency', ascending=False)

# Reset the index for a cleaner DataFrame
title_counts_df = title_counts_df.reset_index(drop=True)

title_counts_df.head()

,Word,Frequency
0,dead,111
1,s,103
2,house,71
3,night,60
4,blood,60


In [10]:
# add to main dataframe 
df['Title Word'] = title_counts_df['Word'] 
df['Title Word Frequency'] = title_counts_df['Frequency']

In [11]:
# Replace "|" with ", " in the "Cast" column
df['Cast'] = df['Cast'].str.replace('|', ', ')

C:\Users\salom\AppData\Local\Temp\ipykernel_20564\865710607.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Cast'] = df['Cast'].str.replace('|', ', ')


In [12]:
# Replace "|" with ", " in the "Cast" column
df['Genres'] = df['Genres'].str.replace('|', ', ')

C:\Users\salom\AppData\Local\Temp\ipykernel_20564\1158279101.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Genres'] = df['Genres'].str.replace('|', ', ')


In [13]:
# Convert all columns with object datatype to string
object_columns = df.select_dtypes(include=['object']).columns
df[object_columns] = df[object_columns].astype(str)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3328 entries, 0 to 3327
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Title                  3328 non-null   object 
 1   Genres                 3328 non-null   object 
 2   Release Date           3328 non-null   object 
 3   Release Country        3328 non-null   object 
 4   Movie Rating           3328 non-null   object 
 5   Review Rating          3076 non-null   float64
 6   Movie Run Time         3328 non-null   object 
 7   Plot                   3328 non-null   object 
 8   Cast                   3328 non-null   object 
 9   Language               3328 non-null   object 
 10  Filming Locations      3328 non-null   object 
 11  Budget                 3328 non-null   object 
 12  Cast Member            3328 non-null   object 
 13  Cast Member Frequency  3328 non-null   int64  
 14  Director               3328 non-null   object 
 15  Desc

# Get Word Freqeuncy of Plot

In [15]:
# Tokenize the text in the "Description" column
words = ' '.join(df['Description']).split()

# Remove punctuation and convert to lowercase (optional)
words = [re.sub(r'[^\w\s]', '', word).lower() for word in words]

# Count the frequency of each word
word_counts = Counter(words)

# Convert the word counts to a DataFrame
word_counts_df = pd.DataFrame(list(word_counts.items()), columns=['Word', 'Frequency'])

# Sort by frequency in descending order
word_counts_df = word_counts_df.sort_values(by='Frequency', ascending=False)

# Reset the index for a cleaner DataFrame
word_counts_df = word_counts_df.reset_index(drop=True)

word_counts_df.head(100)

,Word,Frequency
0,a,6542
1,the,6103
2,with,4310
3,to,3602
4,of,3538
...,...,...
95,being,131
96,there,130
97,strange,130
98,dead,129


In [16]:
df['Description Word'] = word_counts_df['Word'] 
df['Description Word Frequency'] = word_counts_df['Frequency']
df.head()

,Title,Genres,Release Date,Release Country,Movie Rating,Review Rating,Movie Run Time,Plot,Cast,Language,Filming Locations,Budget,Cast Member,Cast Member Frequency,Director,Description,Title Word,Title Word Frequency,Description Word,Description Word Frequency
0,Gut,"Drama, Horror, Thriller",26-Oct-12,USA,nan,3.9,91 min,"Directed by Elias. With Jason Vail, Nicholas W...","Jason Vail, Nicholas Wilder, Sarah Schoofs, Ki...",English,"New York, USA",nan,Lloyd Kaufman,29,Elias,"With Jason Vail, Nicholas Wilder, Sarah Schoo...",dead,111.0,a,6542
1,Haunting Mia Moss,Horror,13-Jan-17,USA,nan,NaN,nan,"Directed by Jake Zelch. With Nicola Fiore, Bri...","Nicola Fiore, Brinke Stevens, Curtis Carnahan,...",English,nan,"$30,000",Eric Roberts,28,Jake Zelch,"With Nicola Fiore, Brinke Stevens, Curtis Car...",s,103.0,the,6103
2,Sleepwalking,Horror,21-Oct-17,Canada,nan,NaN,nan,"Directed by David Briggs. With Alysia Topol, A...","Alysia Topol, Anthony Makela, Kelsi Ashley, Pa...",English,"Sudbury, Ontario, Canada",nan,Debbie Rochon,23,David Briggs,"With Alysia Topol, Anthony Makela, Kelsi Ashl...",house,71.0,with,4310
3,Treasure Chest Horrors II,"Comedy, Horror, Thriller",23-Apr-13,USA,NOT RATED,3.7,82 min,"Directed by M. Kelley, Shawn C. Phillips, Alex...","Veronica Ricci, Nicholas Adam Clark, James Cul...",English,"Baltimore, Maryland, USA",nan,Maria Olsen,21,M,"Kelley, Shawn C. Phillips, Alex Powers. With ...",night,60.0,to,3602
4,Infidus,"Crime, Drama, Horror",10-Apr-15,USA,nan,5.8,80 min,"Directed by Giulio De Santi. With Bonini Mino,...","Bonini Mino, Massimo Caratelli, Maurizio Zaffi...",Italian,nan,nan,Brinke Stevens,18,Giulio De Santi,"With Bonini Mino, Massimo Caratelli, Maurizio...",blood,60.0,of,3538


# Cleaning Budget

In [19]:
# Remove non-numeric characters from the "Budget" column
df['Budget'] = df['Budget'].str.replace(r'[^0-9.]', '', regex=True)

df

,Title,Genres,Release Date,Release Country,Movie Rating,Review Rating,Movie Run Time,Plot,Cast,Language,Filming Locations,Budget,Cast Member,Cast Member Frequency,Director,Description,Title Word,Title Word Frequency,Description Word,Description Word Frequency
0,Gut,"Drama, Horror, Thriller",26-Oct-12,USA,nan,3.9,91 min,"Directed by Elias. With Jason Vail, Nicholas W...","Jason Vail, Nicholas Wilder, Sarah Schoofs, Ki...",English,"New York, USA",,Lloyd Kaufman,29,Elias,"With Jason Vail, Nicholas Wilder, Sarah Schoo...",dead,111.0,a,6542
1,Haunting Mia Moss,Horror,13-Jan-17,USA,nan,NaN,nan,"Directed by Jake Zelch. With Nicola Fiore, Bri...","Nicola Fiore, Brinke Stevens, Curtis Carnahan,...",English,nan,30000,Eric Roberts,28,Jake Zelch,"With Nicola Fiore, Brinke Stevens, Curtis Car...",s,103.0,the,6103
2,Sleepwalking,Horror,21-Oct-17,Canada,nan,NaN,nan,"Directed by David Briggs. With Alysia Topol, A...","Alysia Topol, Anthony Makela, Kelsi Ashley, Pa...",English,"Sudbury, Ontario, Canada",,Debbie Rochon,23,David Briggs,"With Alysia Topol, Anthony Makela, Kelsi Ashl...",house,71.0,with,4310
3,Treasure Chest Horrors II,"Comedy, Horror, Thriller",23-Apr-13,USA,NOT RATED,3.7,82 min,"Directed by M. Kelley, Shawn C. Phillips, Alex...","Veronica Ricci, Nicholas Adam Clark, James Cul...",English,"Baltimore, Maryland, USA",,Maria Olsen,21,M,"Kelley, Shawn C. Phillips, Alex Powers. With ...",night,60.0,to,3602
4,Infidus,"Crime, Drama, Horror",10-Apr-15,USA,nan,5.8,80 min,"Directed by Giulio De Santi. With Bonini Mino,...","Bonini Mino, Massimo Caratelli, Maurizio Zaffi...",Italian,nan,,Brinke Stevens,18,Giulio De Santi,"With Bonini Mino, Massimo Caratelli, Maurizio...",blood,60.0,of,3538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3323,Victor Frankenstein,"Drama, Horror, Sci-Fi, Thriller",25-Nov-15,USA,PG-13,6.0,110 min,Directed by Paul McGuigan. With Daniel Radclif...,"Daniel Radcliffe, Jessica Brown Findlay, Brons...",English,"London, England, UK",40000000,Alexis Kendra,2,Paul McGuigan,"With Daniel Radcliffe, James McAvoy, Jessica ...",nan,NaN,gomez,5
3324,Exorcism Molly Hartley,Horror,9-Oct-15,USA,UNRATED,4.6,96 min,"Directed by Steven R. Monroe. With Sarah Lind,...","Sarah Lind, Devon Sawa, Gina Holden, Peter Mac...",English,"Winnipeg, Manitoba, Canada",,Scott Gillespie,2,Steven R,"Monroe. With Sarah Lind, Devon Sawa, Gina Hol...",nan,NaN,enemies,5
3325,Talon Falls,Horror,13-Oct-17,USA,nan,4.6,75 min,"Directed by Joshua Shreve. With Brad Bell, Jor...","Brad Bell, Jordyn Rudolph, Ryan Rudolph, Morga...",English,nan,,Bailey Gaddis,2,Joshua Shreve,"With Brad Bell, Jordyn Rudolph, Ryan Rudolph,...",nan,NaN,fateful,5
3326,BloodRayne Third Reich,"Action, Adventure, Fantasy, Horror",17-Mar-12,Japan,R,3.0,79 min,"Directed by Uwe Boll. With Natassia Malthe, Br...","Natassia Malthe, Brendan Fletcher, Michael Par...",English,"Zagreb, Croatia",10000000,Anastasia Baranova,2,Uwe Boll,"With Natassia Malthe, Brendan Fletcher, Micha...",nan,NaN,solitary,5


In [20]:
df.to_csv('CleanHorror.csv', index=False)

In [3]:
df = pd.read_csv(r'C:\Users\salom\OneDrive\Área de Trabalho\DSFinal\Horror.csv')

In [4]:
df.head()

,Title,Genres,Release Date,Release Country,Movie Rating,Review Rating,Movie Run Time,Plot,Cast,Language,Filming Locations,Budget,Cast Member,Cast Member Frequency,Director,Title Word,Title Word Frequency,Description Word,Description Word Frequency
0,Gut,"Drama, Horror, Thriller",26-Oct-12,USA,NaN,3.9,91 min,"Directed by Elias. With Jason Vail, Nicholas W...","Jason Vail, Nicholas Wilder, Sarah Schoofs, Ki...",English,"New York, USA",NaN,Lloyd Kaufman,29.0,Elias,deadtime,1.0,young,485.0
1,Haunting Mia Moss,Horror,13-Jan-17,USA,NaN,NaN,NaN,"Directed by Jake Zelch. With Nicola Fiore, Bri...","Nicola Fiore, Brinke Stevens, Curtis Carnahan,...",English,NaN,30000.0,Eric Roberts,28.0,Jake Zelch,huckstabelle,1.0,group,369.0
2,Sleepwalking,Horror,21-Oct-17,Canada,NaN,NaN,NaN,"Directed by David Briggs. With Alysia Topol, A...","Alysia Topol, Anthony Makela, Kelsi Ashley, Pa...",English,"Sudbury, Ontario, Canada",NaN,Debbie Rochon,23.0,David Briggs,nightscape,1.0,friends,326.0
3,Treasure Chest Horrors II,"Comedy, Horror, Thriller",23-Apr-13,USA,NOT RATED,3.7,82 min,"Directed by M. Kelley, Shawn C. Phillips, Alex...","Veronica Ricci, Nicholas Adam Clark, James Cul...",English,"Baltimore, Maryland, USA",NaN,Maria Olsen,21.0,M,blood,61.0,house,322.0
4,Infidus,"Crime, Drama, Horror",10-Apr-15,USA,NaN,5.8,80 min,"Directed by Giulio De Santi. With Bonini Mino,...","Bonini Mino, Massimo Caratelli, Maurizio Zaffi...",Italian,NaN,NaN,Brinke Stevens,18.0,Giulio De Santi,darr,1.0,family,275.0


In [5]:
# Split the "Genres" column by commas and stack the data
stacked_data = df['Genres'].str.split(', ').explode()

# Count the frequency of each word
word_counts = stacked_data.value_counts()

# Create a DataFrame with the word frequencies
word_counts_df = pd.DataFrame({'Word': word_counts.index, 'Frequency': word_counts.values})

word_counts_df

,Word,Frequency
0,Horror,3327
1,Thriller,1378
2,Drama,531
3,Comedy,513
4,Mystery,453
5,Action,336
6,Sci-Fi,309
7,Fantasy,229
8,Crime,121
9,Adventure,116


In [6]:
df['Genres Word'] = word_counts_df['Word']
df['Genres Frequency'] = word_counts_df['Frequency']

In [7]:
df.head()

,Title,Genres,Release Date,Release Country,Movie Rating,Review Rating,Movie Run Time,Plot,Cast,Language,...,Budget,Cast Member,Cast Member Frequency,Director,Title Word,Title Word Frequency,Description Word,Description Word Frequency,Genres Word,Genres Frequency
0,Gut,"Drama, Horror, Thriller",26-Oct-12,USA,NaN,3.9,91 min,"Directed by Elias. With Jason Vail, Nicholas W...","Jason Vail, Nicholas Wilder, Sarah Schoofs, Ki...",English,...,NaN,Lloyd Kaufman,29.0,Elias,deadtime,1.0,young,485.0,Horror,3327.0
1,Haunting Mia Moss,Horror,13-Jan-17,USA,NaN,NaN,NaN,"Directed by Jake Zelch. With Nicola Fiore, Bri...","Nicola Fiore, Brinke Stevens, Curtis Carnahan,...",English,...,30000.0,Eric Roberts,28.0,Jake Zelch,huckstabelle,1.0,group,369.0,Thriller,1378.0
2,Sleepwalking,Horror,21-Oct-17,Canada,NaN,NaN,NaN,"Directed by David Briggs. With Alysia Topol, A...","Alysia Topol, Anthony Makela, Kelsi Ashley, Pa...",English,...,NaN,Debbie Rochon,23.0,David Briggs,nightscape,1.0,friends,326.0,Drama,531.0
3,Treasure Chest Horrors II,"Comedy, Horror, Thriller",23-Apr-13,USA,NOT RATED,3.7,82 min,"Directed by M. Kelley, Shawn C. Phillips, Alex...","Veronica Ricci, Nicholas Adam Clark, James Cul...",English,...,NaN,Maria Olsen,21.0,M,blood,61.0,house,322.0,Comedy,513.0
4,Infidus,"Crime, Drama, Horror",10-Apr-15,USA,NaN,5.8,80 min,"Directed by Giulio De Santi. With Bonini Mino,...","Bonini Mino, Massimo Caratelli, Maurizio Zaffi...",Italian,...,NaN,Brinke Stevens,18.0,Giulio De Santi,darr,1.0,family,275.0,Mystery,453.0


In [8]:
df.to_csv('Horror.csv', index=False)